# 감성 분석

소셜 미디어, 여론조사, 온라인 리뷰, 피드백 등 다양한 분야에서 활용되며 여러 가지 주관적인 단어와 문맥을 기반으로 감성(Sentiment) 수치를 계산하는 방법을 이용한다. 이러한 감성 지수는 긍정, 부정 지수로 구성되며 이를 합산해 긍정 감성 or 부정 감성을 결정한다.  

이러한 감성 분석은 지도학습과 비지도학습 방식으로 나눌 수 있다.  
+ 지도학습: 학습 데이터와 target 레이블 값을 기반으로 감성 분석 학습을 수행한 뒤 이를 기반으로 감성 분석을 예측하는 방법  

+ 비지도학습: 'Lexicon'이라는 일종의 감성 어휘 사전을 이용한다. Lexicon은 감성 분석을 위한 용어와 문맥에 대한 다양한 정보를 가지고 있으며, 이를 이용해 문서의 긍정,부정 감성 여부를 파악한다.

## 지도학습 기반 감성 분석 (IMDB 영화평)

In [23]:
import pandas as pd
# header=0 --> 파일의 첫 번째 줄에 열 이름이 나타나 있다. quoting=3 --> 큰 따옴표를 무시한다. sep=''\t' tap으로 구분해 데이터 출력.
review_df=pd.read_csv("./labeledTrainData.tsv",header=0,sep="\t",quoting=3)
review_df.head(3)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


+ id: 각 데이터의 id
+ sentiment: 1은 긍정, 0은 부정
+ review: 영화평

HTML 형식에서 추출해 br 태그가 존재한다. 따라서 삭제한다. 또한 영어가 아닌 숫자/특수문자 역시 Sentiment를 위한 피처로는 별 의미가 없어보이므로 이를 모두 공란으로 변경한다. 이를 파이썬이 지원하는 re 모듈을 이용해 정규 표현식을 이용하겠다.

In [24]:
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review']=review_df['review'].str.replace('<br />',' ')

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 반환
review_df['review']=review_df['review'].apply( lambda x: re.sub("[^a-zA-Z]"," ",x))


정규표현식의 [^a-zA-Z]의 의미는 영어 대/소문자가 아닌 모든 문자를 찾는 것이다.  
re.sub('패턴', '바꿀문자열', '문자열', 바꿀횟수)

In [25]:
review_df['review']

0         With all this stuff going down at the moment ...
1           The Classic War of the Worlds   by Timothy ...
2         The film starts with a manager  Nicholas Bell...
3         It must be assumed that those who praised thi...
4         Superbly trashy and wondrously unpretentious ...
                               ...                        
24995     It seems like more consideration has gone int...
24996     I don t believe they made this film  Complete...
24997     Guy is a loser  Can t get girls  needs to bui...
24998     This    minute documentary Bu uel made in the...
24999     I saw this movie as a child and it broke my h...
Name: review, Length: 25000, dtype: object

In [26]:
from sklearn.model_selection import train_test_split

In [27]:

from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df=review_df.drop(['id','sentiment'],axis=1, inplace=False)

X_train,X_test,y_train,y_test= train_test_split(feature_df,class_df,test_size=0.3,random_state=156)
print("train data :",X_train.shape,"\n", "test data :" ,X_test.shape)

train data : (17500, 1) 
 test data : (7500, 1)


In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [29]:
# 스톱 워드는 english, filtering, ngram은 (1,2)로 설정해 TF-IDF 벡터화 수행
# LogisticRegression의 C는 10으로 설정.
pipeline=Pipeline([
    ('tfidf_vect',TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))
])

pipeline.fit(X_train['review'],y_train)
pred=pipeline.predict(X_test['review'])
pred_probs=pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f},\tROC-AUC는 {1:.4f}'.format(accuracy_score(y_test,pred),roc_auc_score(y_test,pred_probs)))

C:\Users\kiwon\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


예측 정확도는 0.8936,	ROC-AUC는 0.9598


TF-IDF가 Count 기반에 비해 조금씩 향상된 것을 확인할 수 있다.